In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications import VGG16, ResNet50
from data_utils import CIFAR10Data
from resnet import ResNet18

/home/kitamura/.local/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.2.0-dev20200403). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [2]:
cifar10_data = CIFAR10Data()
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

num_train = int(x_train.shape[0] * 0.8)
num_val = x_train.shape[0] - num_train
mask = list(range(num_train, num_train + num_val))
x_val = x_train[mask]
y_val = y_train[mask]

mask = list(range(num_train))
x_train = x_train[mask]
y_train = y_train[mask]
print(x_train.shape)

data = (x_train, y_train, x_val, y_val, x_test, y_test)

(40000, 32, 32, 3)


In [3]:
weight_decay = 5e-4
lr = 1e-1
num_classes = 10

# model = ResNet50(
#     weights=None,
#     input_shape=x_train.shape[1:],
#     classes=num_classes,
# )
model = ResNet18(
    classes=num_classes,
    input_shape=x_train.shape[1:],
    weight_decay=weight_decay
)

opt = tf.keras.optimizers.SGD(
    learning_rate=lr,
    momentum=0.9,
    nesterov=False
)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "ResNet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [ ]:
from solver import Solver
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

def lr_scheduler(epoch):
    new_lr = lr * (0.1 ** (epoch // 50))
    print("new lf : {}".format(new_lr))
    return new_lr

reduce_lr = LearningRateScheduler(lr_scheduler)

solver = Solver(model, data)
history = solver.train(
    epochs=200, batch_size=128, data_augmentation=True, callbacks=[reduce_lr])

Instructions for updating:
Please use Model.fit, which supports generators.
new lf : 0.1
Epoch 1/200
313/313 [==============================] - 23s 73ms/step - loss: 3.9483 - accuracy: 0.3004 - val_loss: 3.2002 - val_accuracy: 0.3349 - lr: 0.1000
new lf : 0.1
Epoch 2/200
313/313 [==============================] - 22s 70ms/step - loss: 2.5745 - accuracy: 0.4465 - val_loss: 2.2216 - val_accuracy: 0.4910 - lr: 0.1000
new lf : 0.1
Epoch 3/200
313/313 [==============================] - 22s 70ms/step - loss: 1.9153 - accuracy: 0.5398 - val_loss: 1.6866 - val_accuracy: 0.5813 - lr: 0.1000
new lf : 0.1
Epoch 4/200
313/313 [==============================] - 22s 70ms/step - loss: 1.5313 - accuracy: 0.6134 - val_loss: 1.4477 - val_accuracy: 0.6251 - lr: 0.1000
new lf : 0.1
Epoch 5/200
313/313 [==============================] - 22s 70ms/step - loss: 1.3152 - accuracy: 0.6616 - val_loss: 1.5504 - val_accuracy: 0.6109 - lr: 0.1000
new lf : 0.1
Epoch 6/200
313/313 [==============================] - 2